In [2]:
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import arff
import data_processing as dp

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [6]:
data = arff.loadarff("../data/3year.arff")
df = pd.DataFrame(data[0])

In [8]:
X, y = dp.pre_process(df)


AttributeError: 'NoneType' object has no attribute 'corr'